# Introduction
In this tutorial, we will demonstrate how to apply AI segmentation models using CERR on a cohort of patients from the [NCI's IDC](https://portal.imaging.datacommons.cancer.gov/about/) repository.

This example uses lung CT scans of non-small cell lung cancer (NSCLC) patients from the [Lung1](https://wiki.cancerimagingarchive.net/display/Public/NSCLC-Radiomics) dataset.

The entire notebook which includes the download of Octave, CERR, pre-trained model and 5 Lung scans, AI segmentation and visualization takes approximately 15 minutes to run.


## Install GNU Octave and CERR 


* GNU Octave with `statistics`, `io` & `image` packages for Debian/Linux is distributed via MSKCC Box.
* CERR is downloaded from https://github.com/CERR/cerr
* Oct2py and Octave_kernel Python packages are required for Octave-Python bridge.

 *See [installation instructions for different operating systems](https://github.com/stratis-forge/installation-and-dependencies).*  
 
 *Octave and CERR can be saved to Google Drive to avoid downloadig every time.*



In [1]:
# Locations of Octave and CERR
octave_path = '/content/octave'
cerr_path = '/content/CERR'

#### Uncomment and evaluate the cell below to download GNU Octave and CERR

In [2]:
%%capture

# Download Octave
oct_build_box = 'https://mskcc.box.com/shared/static/ylfkha0p66oc8v5kh2z1qx9m13n0ijcx.gz'
oct_save_path = '/content/octave_7.3.0.tar.gz'
! wget {oct_build_box} -O {oct_save_path}
! tar xf {oct_save_path}
! rm {oct_save_path}

# Download CERR
! cd "$(dirname -- "$cerr_path")" && git clone --depth 1 --single-branch --branch octave_dev https://www.github.com/cerr/CERR.git


#### Install dependencies for GNU Octave and Oct2py bridge

In [3]:
%%capture
# Download dependencies for Using Octave
! apt-get update
! cd /usr/lib/x86_64-linux-gnu/ && ln -s libhdf5_serial.so.103 libhdf5_serial.so.100 && ln -s libreadline.so.8 libreadline.so.7
! apt-get install libgraphicsmagick++-q16-12 libcholmod3 libcxsparse3 \
libumfpack5 libspqr2 libqrupdate1 libfftw3-3 libgfortran4 gnuplot openjdk-8-jdk

# Set path to Octave exectuable 
import os, urllib.request, json
os.environ['OCTAVE_EXECUTABLE'] = octave_path + '/bin/octave-cli'
os.environ['PATH'] = octave_path + '/bin:' + os.environ['PATH']

# Install oct2py bridge for Python-Octave communication
! pip3 install octave_kernel
! pip3 install oct2py==5.6.0

# Enable Octave magic
%load_ext oct2py.ipython

## Set up a GCP BigQuery project (Adapted from [IDC tutorials](https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/getting_started/part1_prerequisites.ipynb))





1.   ####  A Google account  
  *Login to existing Google account or see [instructions](https://accounts.google.com/signup/v2/webcreateaccount?dsh=308321458437252901&continue=https%3A%2F%2Faccounts.google.com%2FManageAccount&flowName=GlifWebSignIn&flowEntry=SignUp#FirstName=&LastName=) to create a new account.*

2.   #### Activate GCP for your account and create a GCP project  
 
  1.  Go to https://console.cloud.google.com/, and accept Terms and conditions.
  2.  Click `Select a project` in the upper-left corner of the GCP console.
  3.  Click `Create new project`.
  4.  Open the GCP console menu icon `☰` and select `Dashboard` to view   information about your project. Copy your `Project ID` and insert it in place of `REPLACE_ME_WITH_YOUR_PROJECT_ID` in the cell below.   


3. #### Add the bigquery-public-data project
  1. Open the BigQuery console: https://console.cloud.google.com/bigquery, and click the `+ ADD DATA` button.
  2. Select `Star project by name` and type in `bigquery-public-data` as the project name.


## Authentication for Google services

In [4]:
# Initialize Google Cloud Project ID
my_ProjectID = "lungoarseg" #"REPLACE_ME_WITH_YOUR_PROJECT_ID" #Replace with project ID 

import os
os.environ["GCP_PROJECT_ID"] = my_ProjectID

In [5]:
from google.colab import auth
auth.authenticate_user()

## Download selected cohort

Query the `idc_current` dataset and output a dataframe with URLs for patients from the `RIDER Lung CT` dataset.   

In [6]:
from google.cloud import bigquery

bq_client = bigquery.Client(my_ProjectID) # BigQuery client is initialized with 
                                          # user-input project ID  

selection_query = """
SELECT
  PatientID,
  StudyInstanceUID,
  SeriesInstanceUID,
  collection_id,
  gcs_url
FROM
  `bigquery-public-data.idc_current.dicom_all`
WHERE
  Modality = "CT"
  AND Collection_ID = "nsclc_radiomics"
"""
selection_result = bq_client.query(selection_query)
cohort_df = selection_result.result().to_dataframe()

#cohort_df
display(cohort_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51230 entries, 0 to 51229
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   PatientID          51230 non-null  object
 1   StudyInstanceUID   51230 non-null  object
 2   SeriesInstanceUID  51230 non-null  object
 3   collection_id      51230 non-null  object
 4   gcs_url            51230 non-null  object
dtypes: object(5)
memory usage: 2.0+ MB


None

We will use CT scans of a random subset of 5 patients for this demonstration

In [7]:
from pandas import pandas
import random

num_sample = 5
all_patients = set(cohort_df["PatientID"])
sample_patients = random.sample(all_patients,num_sample)
subset_df = cohort_df[cohort_df["PatientID"].isin(sample_patients)]

selected_df = pandas.DataFrame()
for patient in sample_patients:
  patient_df = subset_df[subset_df["PatientID"]==patient]
  study_uids = list(patient_df["StudyInstanceUID"].unique())
  patient_study_df = patient_df[patient_df["StudyInstanceUID"]==study_uids[0]]
  series_uids = list(patient_study_df["SeriesInstanceUID"].unique())
  patient_series_df = patient_study_df[patient_study_df["SeriesInstanceUID"]==series_uids[0]]
  patient_series_df = patient_series_df.reset_index(drop=True)
  if patient==sample_patients[0]:
    selected_df = patient_series_df
  else:
    selected_df = pandas.concat([selected_df,patient_series_df])

display(selected_df)

,PatientID,StudyInstanceUID,SeriesInstanceUID,collection_id,gcs_url
0,LUNG1-175,1.3.6.1.4.1.32722.99.99.2762427841906089383497...,1.3.6.1.4.1.32722.99.99.2344980020185865130145...,nsclc_radiomics,gs://idc-open-cr/5226861a-7519-4132-ad46-cd86f...
1,LUNG1-175,1.3.6.1.4.1.32722.99.99.2762427841906089383497...,1.3.6.1.4.1.32722.99.99.2344980020185865130145...,nsclc_radiomics,gs://idc-open-cr/bae88257-b9e8-4c79-bb06-83200...
2,LUNG1-175,1.3.6.1.4.1.32722.99.99.2762427841906089383497...,1.3.6.1.4.1.32722.99.99.2344980020185865130145...,nsclc_radiomics,gs://idc-open-cr/ad4e2b80-7c6d-4798-a512-760eb...
3,LUNG1-175,1.3.6.1.4.1.32722.99.99.2762427841906089383497...,1.3.6.1.4.1.32722.99.99.2344980020185865130145...,nsclc_radiomics,gs://idc-open-cr/b6f4a0c5-6ac1-4650-ba2f-db2be...
4,LUNG1-175,1.3.6.1.4.1.32722.99.99.2762427841906089383497...,1.3.6.1.4.1.32722.99.99.2344980020185865130145...,nsclc_radiomics,gs://idc-open-cr/0073ab28-3acf-4aa3-85b8-3ac51...
...,...,...,...,...,...
129,LUNG1-206,1.3.6.1.4.1.32722.99.99.1603870615197838322108...,1.3.6.1.4.1.32722.99.99.6753366146575328532708...,nsclc_radiomics,gs://idc-open-cr/835da998-cef6-49b5-ae84-455e7...
130,LUNG1-206,1.3.6.1.4.1.32722.99.99.1603870615197838322108...,1.3.6.1.4.1.32722.99.99.6753366146575328532708...,nsclc_radiomics,gs://idc-open-cr/3bef325c-ea30-43d6-b069-aba0b...
131,LUNG1-206,1.3.6.1.4.1.32722.99.99.1603870615197838322108...,1.3.6.1.4.1.32722.99.99.6753366146575328532708...,nsclc_radiomics,gs://idc-open-cr/7dbaf7a3-1604-4835-a3d1-b1396...
132,LUNG1-206,1.3.6.1.4.1.32722.99.99.1603870615197838322108...,1.3.6.1.4.1.32722.99.99.6753366146575328532708...,nsclc_radiomics,gs://idc-open-cr/98fe4bb4-c5be-40f5-bf08-c1c89...


Save URLs into a manifest file for download using `gsutil` from the Google Cloud SDK*. 

*Note: Google Cloud SDK is pre-installed on Colab, but will need to be installed if downloading directly to your computer.* 

In [8]:
%%capture
#Save URLs to manifest file
selected_df["gcs_url"].to_csv("manifest.txt", header=False, index=False)

#Download images to /content/downloaded_cohort_files
!rm -rf downloaded_cohort_files && mkdir downloaded_cohort_files
!cat manifest.txt | gsutil -m cp -I downloaded_cohort_files

## Install *DICOMSort* to organize downloaded data



In [9]:
%%capture
!git clone https://github.com/pieper/dicomsort src/dicomsort
!pip install -r /content/src/dicomsort/requirements.txt

Organize DICOM files by patient ID and modality

In [10]:
%%capture
# Run  DICOMSort 
!python src/dicomsort/dicomsort.py -u /content/downloaded_cohort_files/ /content/organized_cohort_files/dicom/%PatientID/%SOPInstanceUID.dcm

# Delete temporary directory for unsorted DICOM data 
!rm -rf /content/downloaded_cohort_files/

## Download and unpack pretrained model
(requires about 5 minutes to download)

In [11]:
# Location to downloaded LungOAR model
lung_model_dir = '/content/pretrained_lungOAR_model'


In [12]:
%%capture
# Download packaged environment for the AI model
ct_lung_oar_model_box = 'https://mskcc.box.com/shared/static/dqt4sm6laggmuylrrvn1i8nlt7b6hptt.gz'
! mkdir {lung_model_dir}
ct_lung_oar_save_path = 'ct_lung_oar.tar.gz'
! wget {ct_lung_oar_model_box} -O {ct_lung_oar_save_path}
! tar xf {ct_lung_oar_save_path} -C {lung_model_dir}
! rm {ct_lung_oar_save_path}

## Add CERR to GNU Octave path and load required Octave packages

In [13]:
%octave_push cerr_path

In [14]:
%%capture
%%octave

# Load the required Octave packages
pkg load statistics
pkg load image
pkg load io

# Add CERR to path
addpath(cerr_path)
addToPath(cerr_path)

### Convert DIOCM data to CERR planC format

In [15]:
%%capture
%%octave
sourceDir = '/content/organized_cohort_files/dicom'  
destDir = '/content/organized_cohort_files/cerr/'  
zipFlag = 'No';
mergeFlag = 'No';
singleCerrFileFlag = 'No';
init_ML_DICOM;
batchConvertWithSubDirs(sourceDir,destDir,zipFlag,mergeFlag,singleCerrFileFlag)

# Apply segmentation model to data in planC format

In [16]:
%octave_push lung_model_dir

In [17]:
%%capture
%%octave

#Run segmentation model
src_dir = '/content/organized_cohort_files/cerr';
cmd_flag = 'condaEnv';
new_session = true;

#Loop over CERR files
dirS = dir([src_dir,filesep,'*.mat']);
dirS([dirS.isdir]) = [];
origScanNumV = nan(1,length(dirS));
fileC = fullfile(src_dir,{dirS.name});
numFiles = length(fileC)

## Auto-segmentation model path and session dir
#model_dir = '/content/pretrained_lungOAR_model';
algorithm = 'CT_LungOAR_incrMRRN';
sessionPath = '/content/temp_session';
cmdFlag = 'condaEnv';
newSessionFlag = 1;

for iFile = 1:numFiles
    # Load planC
    planC = loadPlanC(fileC{iFile}, tempdir);
    planC = updatePlanFields(planC);
    planC = quality_assure_planC(fileC{iFile}, planC);
    
    # AI-segment
    planC = runAIforPlanC(planC,sessionPath,algorithm,cmdFlag,...
        newSessionFlag,[],[],lung_model_dir);

    # Save planC with AI segmentation
    planC = save_planC(planC,[],'passed',fileC{iFile});

end



# Extract segmented masks from planC

In [18]:
%%capture
%%octave

%Load sample result
out_path = fullfile(src_dir,dirS(1).name)
planC = loadPlanC(out_path,tempdir);
planC = updatePlanFields(planC);
planC = quality_assure_planC(out_path,planC);

%Get scan array
indexS = planC{end};
scanNum = 1;
ctOffset = planC{indexS.scan}(scanNum).scanInfo(1).CTOffset;
scanArray = single(getScanArray(scanNum,planC)) - ctOffset;

%Get structure labels & masks
numStructs = length(planC{indexS.structures});
structNameC = {planC{indexS.structures}.structureName};
strNameC = {'Lung_Left', 'Lung_Right', 'Heart',...
            'Esophagus', 'Cord', 'PBT'};

for strNum = 1:length(strNameC)
    searchStr = strNameC{strNum};
    idx = getMatchingIndex(searchStr,structNameC,'EXACT');
    mask3M = getStrMask(idx(end), planC);
    maskC{strNum} = mask3M;
end     

% Get scan grid
[xScanValsV,yScanValsV,zScanValsV] = getScanXYZVals(planC{indexS.scan}(scanNum));


# Visualize results

In [21]:
%octave_pull maskC strNameC scanArray xScanValsV yScanValsV zScanValsV

In [22]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import ipywidgets as widgets
from ipywidgets import Layout

window_center = -500
window_width = 1500

extent_trans = np.min(xScanValsV), np.max(xScanValsV), np.min(yScanValsV), np.max(yScanValsV)
extent_sag = np.min(yScanValsV), np.max(yScanValsV), np.min(zScanValsV), np.max(zScanValsV)
extent_cor = np.min(xScanValsV), np.max(xScanValsV), np.min(zScanValsV), np.max(zScanValsV)

img_siz = scanArray.shape
clear_output(wait=True)

def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    
    return window_image

def rotate_image(img):
    return(list(zip(*img)))

def show_slice(slcNum, view):
    clear_output(wait=True)
    print(view + ' view slice ' + str(slcNum))
    if 'fig' in locals():
        fig.remove()
    fig, (ax,ax_legend) = plt.subplots(1,2)
    ax_legend.set_visible(False)
    
    if view.lower() == 'axial':
        windowed_img = window_image(scanArray[: ,: ,slcNum - 1], window_center, window_width)
        extent = extent_trans
    elif view.lower() == 'sagittal':
        #windowed_img = rotate_image(window_image(scanArray[40:210, slcNum + 79, :], window_center, window_width))
        windowed_img = rotate_image(window_image(scanArray[:, slcNum - 1, :], window_center, window_width))
        extent = extent_sag
    elif view.lower() == 'coronal':
        windowed_img = rotate_image(window_image(scanArray[slcNum - 1, :, :], window_center, window_width))
        extent = extent_cor
    else:
        raise Exeception('Invalid view type: ' + view)
    
    im1 = ax.imshow(windowed_img, cmap=plt.cm.gray, alpha=1,
                    interpolation='nearest', extent=extent)
    
    cmaps = [plt.cm.get_cmap("Oranges").copy(),plt.cm.get_cmap("Oranges").copy(), \
             plt.cm.get_cmap("Blues").copy(),plt.cm.get_cmap("Blues").copy(), \
             plt.cm.get_cmap("Purples").copy(),plt.cm.get_cmap("Greens").copy()]

    if view.lower() == 'axial':
        for maskNum in range(0,6,1):
            mask_cmap = cmaps[maskNum]
            mask_cmap.set_under('k', alpha=0)
            im2 = ax.imshow(maskC[0,maskNum][:,:,slcNum-1], 
                            cmap=mask_cmap, alpha=.8, extent=extent,
                            interpolation='none', clim=[0.5, 1])      
            
    elif view.lower() == 'sagittal': 
        for maskNum in range(0,6,1):
            mask_cmap = cmaps[maskNum]
            mask_cmap.set_under('k', alpha=0)
            #im2 = ax.imshow(rotate_image(maskC[0,maskNum][50:200, slcNum + 79, :]),
            #                cmap=mask_cmap, alpha=.8, extent=extent,
            #                interpolation='none', clim=[0.5, 1])
            im2 = ax.imshow(rotate_image(maskC[0,maskNum][:, slcNum - 1, :]),
                            cmap=mask_cmap, alpha=.8, extent=extent,
                            interpolation='none', clim=[0.5, 1])
            
    elif view.lower() == 'coronal':
        for maskNum in range(0,6,1):
            mask_cmap = cmaps[maskNum]
            mask_cmap.set_under('k', alpha=0)
            #im2 = ax.imshow(rotate_image(maskC[0,maskNum][slcNum + 60, 50:200, :]),
            #                cmap=mask_cmap, alpha=.8, extent=extent,
            #                interpolation='none', clim=[0.5, 1])
            im2 = ax.imshow(rotate_image(maskC[0,maskNum][slcNum - 1, :, :]),
                            cmap=mask_cmap, alpha=.8, extent=extent,
                            interpolation='none', clim=[0.5, 1])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    plt.rcParams["figure.figsize"] = (5, 3)
    plt.show()
    
slice_slider_axial = widgets.IntSlider(min=1,max=img_siz[2],step=1)
outputSlc_axial = widgets.Output()
    
#slice_slider_sagittal = widgets.IntSlider(min=1,max=88,step=1)
slice_slider_sagittal = widgets.IntSlider(min=1,max=img_siz[1],step=1)
outputSlc_sagittal = widgets.Output()

#slice_slider_coronal = widgets.IntSlider(min=1,max=116,step=1)
slice_slider_coronal = widgets.IntSlider(min=1,max=img_siz[0],step=1)
outputSlc_coronal = widgets.Output()

display(slice_slider_axial, outputSlc_axial, slice_slider_sagittal, outputSlc_sagittal, slice_slider_coronal,outputSlc_coronal)

def update_slice_axial(change):
    with outputSlc_axial:
        show_slice(change['new'], 'axial')

def update_slice_sagittal(change):
    with outputSlc_sagittal:
        show_slice(change['new'], 'sagittal')
        
def update_slice_coronal(change):
    with outputSlc_coronal:
        show_slice(change['new'], 'coronal')
        
slice_slider_axial.observe(update_slice_axial, names='value')
slice_slider_sagittal.observe(update_slice_sagittal, names='value')
slice_slider_coronal.observe(update_slice_coronal, names='value')

# Set values to display figures
slice_slider_axial.value = round(img_siz[2]/2)
slice_slider_sagittal.value = round(img_siz[1]/2)
slice_slider_coronal.value = round(img_siz[0]/2)

IntSlider(value=1, max=135, min=1)

Output()

IntSlider(value=1, max=512, min=1)

Output()

IntSlider(value=1, max=512, min=1)

Output()